# Fit DSM to low resolution initial DEM

This notebook details how to estimate and apply the transform to fit A DSM to the low resolution initial DEM. This method is currently implemented in cars.

## Notebook parameters

Those parameters have to be completed to use the notebook.

In [ ]:
# Path to the cars folder
cars_home = "TODO"
# Path to the directory containing the content.json file of the prepare step output
content_dir = "TODO"

## Imports

In [ ]:
### Trick to override cars verision
import sys
sys.path = [cars_home] + sys.path
import os
os.environ['OTB_APPLICATION_PATH'] = os.path.join(cars_home,'build','lib','otb','applications')+':'+os.environ['OTB_APPLICATION_PATH']
###
# Silent OTB info logs
os.environ['OTB_LOGGER_LEVEL']='WARNING'
import xarray as xr
import scipy as sp
import numpy as np
import math
import matplotlib.pyplot as plt
from cars.core import projection
from cars.steps import triangulation
from cars.steps import rasterization
from cars.conf import output_prepare
from cars.core import constants as cst
from scipy.signal import butter, lfilter, filtfilt, lfilter_zi

# Reading input data

In [ ]:
lowres_dsm_from_matches = xr.open_dataset(os.path.join(content_dir,'lowres_dsm_from_matches.nc'))
lowres_initial_dem = xr.open_dataset(os.path.join(content_dir,'lowres_initial_dem.nc'))
conf = output_prepare.read_preprocessing_content_file(os.path.join(content_dir,'content.json'))
img1 = conf['input']['img1']
img2 = conf['input']['img2']
dem = conf['input']['srtm_dir']
matches = np.load(os.path.join(content_dir,'matches.npy'))
disp_to_alt_ratio = conf['preprocessing']['output']['disp_to_alt_ratio']
srtm_dir = conf['input']['srtm_dir']

## Look for direction of oscillations

We compute the direction in which oscillations might happen (increasing acquisition time direction)

In [ ]:
vec1 = projection.get_time_ground_direction(img1,dem=srtm_dir)
vec2 = projection.get_time_ground_direction(img2, dem=srtm_dir)
time_direction_vector = (vec1+vec2)/2
print("Direction img1 {} degree wrt horizontal axis<".format(180*math.atan2(vec1[1], vec1[0])/math.pi))
print("Direction img2 {} degree wrt horizontal axis".format(180*math.atan2(vec2[1], vec2[0])/math.pi))
print("Oscillation direction: {} ({} degree wrt horizontal axis)".format(time_direction_vector,180*math.atan2(time_direction_vector[1], time_direction_vector[0])/math.pi))

## Measuring initial difference with low resolution DEM

In [ ]:
dsm_diff = lowres_initial_dem[cst.RASTER_HGT]-lowres_dsm_from_matches[cst.RASTER_HGT]
(dsm_diff).plot(robust=True)

## Extracting difference signal (offset and oscillations)

We project differences on the  `(x,y), time_direction_vector` axis.

In [ ]:
origin = [float(dsm_diff[cst.X][0].values), float(dsm_diff[cst.Y][0].values)]
x_values_2d, y_values_2d = np.meshgrid(dsm_diff[cst.X], dsm_diff[cst.Y])
curv_coords = projection.project_coordinates_on_line(x_values_2d, y_values_2d, origin, time_direction_vector)
curv_array = xr.DataArray(dsm_diff.values.ravel(), coords={"curv" : curv_coords.ravel()}, dims = ("curv"))
curv_array = curv_array.dropna(dim='curv')
curv_array = curv_array.sortby('curv')
curv_array.plot(figsize=(15,5))

The we perform denoising by aggregating median along time axis.

In [ ]:
min_curv = np.min(curv_array.curv)
max_curv = np.max(curv_array.curv)
nbins = int(math.ceil((max_curv-min_curv)/(lowres_dsm_from_matches.attrs[cst.RESOLUTION])))
filtered_curv_array = curv_array.groupby_bins('curv',nbins).median()
filtered_curv_array = filtered_curv_array.rename({'curv_bins': 'curv'})
filtered_curv_array = filtered_curv_array.assign_coords({'curv' : np.array([d.mid for d in filtered_curv_array.curv.data])})

We also compute the number of point in each median slot, and discard measurements for slots with insufficient number of points (< 100)

In [ ]:
filtered_curv_npoints = curv_array.groupby_bins('curv',nbins).count()
filtered_curv_npoints = filtered_curv_npoints.rename({'curv_bins': 'curv'})
filtered_curv_npoints = filtered_curv_npoints.assign_coords({'curv' : np.array([d.mid for d in filtered_curv_npoints.curv.data])})
filtered_curv_array = filtered_curv_array.where(filtered_curv_npoints > 100).dropna(dim='curv')

Next, we will apply butterworth low pass filtering to extract low frequency of the difference signal.

In [ ]:
b, a = butter(3, 0.05)
zi = lfilter_zi(b, a)
z, _ = lfilter(b, a, filtered_curv_array.values, zi=zi*filtered_curv_array.values[0])
z2, _ = lfilter(b, a, z, zi=zi*z[0])
filtered_curv_array_lowpass = xr.DataArray(filtfilt(b, a,filtered_curv_array.values),coords=filtered_curv_array.coords)

Display both curves (median difference and low passed filtered low pass median difference)

In [ ]:
xr.concat([filtered_curv_array,filtered_curv_array_lowpass],dim='tmp').isel(tmp=[0,1]).plot(hue='tmp',figsize=(15,5))

## Smoothed difference modelling
We will use cubic splines to model this smoothed difference signal. Our aim is to find the smoothest cubic splines that have a RMSE < 0.3 meters.

In [ ]:
# Initialize s parameter
best_s = 100*len(filtered_curv_array_lowpass.curv)

# Compute first spline and RMSE
splines = sp.interpolate.UnivariateSpline(filtered_curv_array_lowpass.curv,filtered_curv_array_lowpass.values, ext=3, k=3, s=best_s)
estimated_correction = xr.DataArray(splines(filtered_curv_array_lowpass.curv),coords=filtered_curv_array_lowpass.coords)
rmse = (filtered_curv_array_lowpass-estimated_correction).std(dim='curv')

# Loop to find best s
s = [best_s]
rmses = [rmse]
target_rmse = 0.3
while rmse > target_rmse and best_s > 0.001:
    best_s/=2.
    splines = sp.interpolate.UnivariateSpline(filtered_curv_array_lowpass.curv,filtered_curv_array_lowpass.values, ext=3, k=3, s=best_s)
    estimated_correction = xr.DataArray(splines(filtered_curv_array_lowpass.curv),coords=filtered_curv_array_lowpass.coords)
    rmse = (filtered_curv_array_lowpass-estimated_correction).std(dim='curv')
    s.append(best_s)
    rmses.append(rmse)

# Display
fig, ax = plt.subplots()
ax.set_xlabel('value for s')
ax.set_ylabel('rmse')
ax.plot(rmses)
print("Best smoothing factor: {} (rmse = {} meters)".format(best_s, rmse.values))

Now we can interpolate the spline to obtain the 1D correction

In [ ]:
splines = sp.interpolate.UnivariateSpline(filtered_curv_array_lowpass.curv,filtered_curv_array_lowpass.values, ext=3, k=3, s = best_s)
estimated_correction = xr.DataArray(splines(filtered_curv_array.curv),coords=filtered_curv_array.coords)
xr.concat([filtered_curv_array, filtered_curv_array_lowpass, estimated_correction, filtered_curv_array-estimated_correction],dim='tmp').isel(tmp=[0,1,2,3]).plot(hue='tmp',figsize=(15,5))

Likewise, we can interpolate the splines in 2D to get the 2D z correction field

In [ ]:
estimated_correction_2d = xr.DataArray(splines(curv_coords),coords=dsm_diff.coords)
xr.concat((dsm_diff,estimated_correction_2d, lowres_initial_dem[cst.RASTER_HGT] - lowres_dsm_from_matches[cst.RASTER_HGT] - estimated_correction_2d), dim='tmp').plot(col='tmp', robust=True, figsize=(15,4))

The mean column is also visualized (before and after) as well as the correction (in the center).

## Applying correction in triangulation

In this section we will demonstrate how to use the correction during triangulation, using matches triangulation. first we compute the initial points cloud from matches

In [ ]:
initial_points_cloud = triangulation.triangulate_matches(conf,matches)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science library notebook is not installed in interpreter Python 3.6.9 64-bit.

We can now estimate z correction for each triangulated point using our splines

In [ ]:
initial_points_cloud_z_correction = splines(projection.project_coordinates_on_line(initial_points_cloud[cst.X], initial_points_cloud[cst.Y], origin, time_direction_vector))

We can then convert this z correction into a disparity correction:

In [ ]:
disp_correction = initial_points_cloud_z_correction/disp_to_alt_ratio

Apply this disparity correction to matches:

In [ ]:
corrected_matches = np.copy(matches)
corrected_matches[:,2]= corrected_matches[:,2]-disp_correction[:,0]

And triangulate again:

In [ ]:
corrected_points_cloud = triangulation.triangulate_matches(conf,corrected_matches)

Now we can rasterize corrected cloud.

In [ ]:
startx = float(np.min(lowres_dsm_from_matches[cst.X]).values)-0.5*lowres_dsm_from_matches.resolution
starty = float(np.max(lowres_dsm_from_matches[cst.Y]).values)+0.5*lowres_dsm_from_matches.resolution
sizex = lowres_dsm_from_matches[cst.X].shape[0]
sizey = lowres_dsm_from_matches[cst.Y].shape[0]
corrected_lowres_dsm_from_matches = rasterization.simple_rasterization_dataset([corrected_points_cloud],lowres_dsm_from_matches.resolution,xstart=startx, ystart=starty, xsize=sizex, ysize=sizey, epsg=4326)

Last, we display diff with low resolution intial dem:

In [ ]:
xr.concat((lowres_initial_dem[cst.RASTER_HGT]-lowres_dsm_from_matches[cst.RASTER_HGT],
           lowres_initial_dem[cst.RASTER_HGT]-corrected_lowres_dsm_from_matches[cst.RASTER_HGT]), dim='tmp').plot(col='tmp', robust=True,figsize=(10,4))